<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Apache_Solr_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and run Solr on a Google Colab instance


In [51]:
!java -version

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [52]:
!wget -nc https://downloads.apache.org/lucene/solr/8.10.1/solr-8.10.1.tgz
!tar xf solr-8.10.1.tgz

File ‘solr-8.10.1.tgz’ already there; not retrieving.



In [53]:
!(cd solr-8.10.1; ./bin/solr start -c -force)


Port 8983 is already being used by another process (pid: 189)
Please choose a different port using the -p option.



In [54]:
!(cd solr-8.10.1; ./bin/solr create_collection -c test_collection -force)

         To turn off: bin/solr config -c test_collection -p 8983 -action set-user-property -property update.autoCreateFields -value false
WARN  - 2021-10-20 05:56:43.637; org.apache.zookeeper.ClientCnxn; An exception was thrown while closing send thread for session 0x10000028fce0002. => EndOfStreamException: Unable to read additional data from server sessionid 0x10000028fce0002, likely server has closed socket
	at org.apache.zookeeper.ClientCnxnSocketNIO.doIO(ClientCnxnSocketNIO.java:77)
org.apache.zookeeper.ClientCnxn$EndOfStreamException: Unable to read additional data from server sessionid 0x10000028fce0002, likely server has closed socket
	at org.apache.zookeeper.ClientCnxnSocketNIO.doIO(ClientCnxnSocketNIO.java:77) ~[zookeeper-3.6.2.jar:3.6.2]
	at org.apache.zookeeper.ClientCnxnSocketNIO.doTransport(ClientCnxnSocketNIO.java:350) ~[zookeeper-3.6.2.jar:3.6.2]
	at org.apache.zookeeper.ClientCnxn$SendThread.run(ClientCnxn.java:1275) ~[zookeeper-3.6.2.jar:3.6.2]

ERROR: 
Collection 'te

In [55]:
!curl http://localhost:8983/solr/admin/collections?action=CLUSTERSTATUS


{
  "responseHeader":{
    "status":0,
    "QTime":3},
  "cluster":{
    "collections":{
      "test_collection":{
        "pullReplicas":"0",
        "replicationFactor":"1",
        "shards":{"shard1":{
            "range":"80000000-7fffffff",
            "state":"active",
            "replicas":{"core_node2":{
                "core":"test_collection_shard1_replica_n1",
                "node_name":"172.28.0.2:8983_solr",
                "base_url":"http://172.28.0.2:8983/solr",
                "state":"active",
                "type":"NRT",
                "force_set_state":"false",
                "leader":"true"}},
            "health":"GREEN"}},
        "router":{"name":"compositeId"},
        "maxShardsPerNode":"-1",
        "autoAddReplicas":"false",
        "nrtReplicas":"1",
        "tlogReplicas":"0",
        "health":"GREEN",
        "znodeVersion":4,
        "configName":"test_collection"}},
    "live_nodes":["172.28.0.2:8983_solr"]}}


Expose solr using ngrok

In [65]:
!wget -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

# !(sh -c "nohup ./ngrok http 8983 &") ; echo "started ngrok"1
# !curl http://localhost:4040/api/tunnels

File ‘ngrok-stable-linux-amd64.zip’ already there; not retrieving.

Archive:  ngrok-stable-linux-amd64.zip


In [70]:
%pip install pysolr

In [77]:
import pysolr

solr = pysolr.Solr('http://localhost:8983/solr/test_collection')

In [79]:
solr.add([{'id':1,'text':'hello'}])


'{\n  "responseHeader":{\n    "rf":1,\n    "status":0,\n    "QTime":459}}\n'

In [83]:
results = solr.search('hello')
results.raw_response

{'response': {'docs': [], 'numFound': 0, 'numFoundExact': True, 'start': 0},
 'responseHeader': {'QTime': 1,
  'params': {'q': 'hello', 'wt': 'json'},
  'status': 0,
  'zkConnected': True}}